In [ ]:
%pip install -e ..
%restart_python

In [ ]:
from typing import Dict
import math
import argparse
from mlflow import MlflowClient
from loguru import logger
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
# ------------------------------------------------------------------------------
# Databricks Context
# ------------------------------------------------------------------------------
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

In [ ]:
# # ---------------------------------------------------------
# # Args
# # ---------------------------------------------------------

# parser = argparse.ArgumentParser(description="Deploy or update Databricks model serving endpoint")

# parser.add_argument(
#     "--candidate_run_id",
#     type=str,
#     # default="honeywell_mlops_dev.honeywell.ev_battery_charging_model_basic",
#     required=True,
#     help="Fully-qualified UC model name (e.g. catalog.schema.model_name)",
# )

# parser.add_argument(
#     "--model_name",
#     type=str,
#     required=True,
#     help="Model name",
# )


# # args = parser.parse_args()
# args, unknown = parser.parse_known_args()
# if unknown:
#     logger.info("Ignoring unknown args: %s", unknown)

In [ ]:
# ---------------------------------------------------------
# Widgets (Notebook Arguments)
# ---------------------------------------------------------
# Define the widgets (these act as your "arguments")
dbutils.widgets.text("candidate_run_id", "")
dbutils.widgets.text("model_name", "")

# Retrieve values into variables
arg_candidate_run_id = dbutils.widgets.get("candidate_run_id")
arg_model_name = dbutils.widgets.get("model_name")

# Basic validation (similar to 'required=True')
if not arg_candidate_run_id or not arg_model_name:
    raise ValueError("Missing required parameters: candidate_run_id or model_name")

logger.info(f"Promoting model: {arg_model_name} from run: {arg_candidate_run_id}")


In [ ]:

# ------------------------------------------------------------------------------
# Databricks Context
# ------------------------------------------------------------------------------
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)
def promotion_gate(
    model_name: str,
    candidate_run_id: str,
    improvement_threshold: float = 0.0,
    min_auc_delta: float = -0.002,
    required_metrics: Dict[str, str] = None,
) -> bool:
    """
    Decide whether a candidate model should be promoted to Champion.

    Rules:
    - If no Champion exists → auto-promote
    - ROC-AUC must not regress beyond min_auc_delta
    - F1 must improve by at least improvement_threshold

    Returns:
        True  → promotion allowed
        False → promotion blocked
    """

    client = MlflowClient()

    # ------------------------------------------------------------------
    # Resolve candidate metrics
    # ------------------------------------------------------------------
    try:
        candidate_run = client.get_run(candidate_run_id)
    except Exception as exc:
        raise RuntimeError(f"Invalid candidate_run_id: {candidate_run_id}") from exc

    cand_metrics = candidate_run.data.metrics or {}

    # Default metric keys (overrideable)
    if required_metrics is None:
        required_metrics = {
            "f1": "f1_score",
            "auc": "roc_auc",
        }

    # ------------------------------------------------------------------
    # Resolve Champion baseline
    # ------------------------------------------------------------------
    try:
        mv = client.get_model_version_by_alias(model_name, "champion")
        champion_run_id = mv.run_id
        champion_run = client.get_run(champion_run_id)
        champ_metrics = champion_run.data.metrics or {}
    except Exception:
        logger.info("No Champion found. Auto-promoting first model.")
        return True

    # ------------------------------------------------------------------
    # Extract metrics
    # ------------------------------------------------------------------
    f1_new = cand_metrics.get(required_metrics["f1"])
    f1_old = champ_metrics.get(required_metrics["f1"])

    auc_new = cand_metrics.get(required_metrics["auc"])
    auc_old = champ_metrics.get(required_metrics["auc"])

    # ------------------------------------------------------------------
    # Validate metric presence + sanity
    # ------------------------------------------------------------------
    missing = []
    for name, value in {
        "f1_new": f1_new,
        "f1_old": f1_old,
        "auc_new": auc_new,
        "auc_old": auc_old,
    }.items():
        if value is None or not isinstance(value, (int, float)) or math.isnan(value):
            missing.append(name)

    if missing:
        raise RuntimeError(
            f"Missing or invalid metrics for promotion gate: {missing}. "
            f"Candidate metrics={cand_metrics}, Champion metrics={champ_metrics}"
        )

    # ------------------------------------------------------------------
    # Compute deltas
    # ------------------------------------------------------------------
    delta_f1 = f1_new - f1_old
    delta_auc = auc_new - auc_old

    logger.info(
        "Promotion gate metrics → "
        "F1: new=%.6f old=%.6f Δ=%.6f | "
        "AUC: new=%.6f old=%.6f Δ=%.6f | "
        "thresholds: ΔF1>=%.6f, ΔAUC>=%.6f",
        f1_new, f1_old, delta_f1,
        auc_new, auc_old, delta_auc,
        improvement_threshold, min_auc_delta,
    )

    # ------------------------------------------------------------------
    # Gate 1 — ROC-AUC regression guardrail
    # ------------------------------------------------------------------
    if delta_auc < min_auc_delta:
        logger.warning(
            "❌ Promotion rejected: ROC-AUC regressed too much "
            "(Δ=%.6f < min allowed %.6f)",
            delta_auc, min_auc_delta
        )
        return False

    # ------------------------------------------------------------------
    # Gate 2 — F1 improvement requirement
    # ------------------------------------------------------------------
    if delta_f1 < improvement_threshold:
        logger.warning(
            "❌ Promotion rejected: F1 improvement too small "
            "(Δ=%.6f < required %.6f)",
            delta_f1, improvement_threshold
        )
        return False

    # ------------------------------------------------------------------
    # PASS
    # ------------------------------------------------------------------
    logger.info("✅ Promotion gate PASSED")
    return True

In [ ]:
from promote_champ_to_challenger import promote_challenger_to_champion


# allowed_model = promotion_gate()
# promote_challenger_to_champion(model_name="honeywell_mlops_dev.honeywell.ev_battery_charging_model_basic", allow_promotion=allowed_model)
# # dbutils.setOutput("allowed_model", allowed_model)

In [ ]:
def run_promotion_flow(
    model_name: str,
    candidate_run_id: str,
) -> None:
    logger.info("Starting promotion flow for model: %s", model_name)

    should_promote = promotion_gate(
        model_name=model_name,
        candidate_run_id=candidate_run_id,
        improvement_threshold=0.0,
        min_auc_delta=0.0,
    )
    
    if not should_promote:
        logger.info("🚫 Promotion gate BLOCKED. No alias changes will be made.")
        return

    logger.info("🚀 Promotion gate PASSED. Swapping aliases.")

    promote_challenger_to_champion(
        model_name=model_name,
        allow_promotion=True,   # 🔐 explicit authorization
    )

    logger.info("🎯 Promotion completed successfully.")
    return True


In [ ]:
run_promotion_flow(
    model_name=arg_model_name,
    candidate_run_id=arg_candidate_run_id,
)